In [53]:
# import sys
# sys.path.append("../")
from lib.clean import standardize_desc_cols, clean_column_names
from lib.uid import gen_uid
import re

In [54]:
import pandas as pd
from datamatch import (
    JaroWinklerSimilarity,
    ThresholdMatcher,
    ColumnsIndex,
)

In [55]:
def read_data():
    df = pd.read_csv("../data/OK/aggregate_members_oathkeepers_cleaned.csv")
    return df 

In [56]:
dfa = read_data()

In [57]:
dfa = dfa.fillna("")

In [58]:
dfa.loc[:, "state"] = dfa.state.str.lower().str.strip().str.replace(r"ma\.", "ma", regex=True)

In [59]:
dfa.columns

Index(['id_number', 'group_name', 'bool', 'city', 'state', 'email',
       'undefined_date', 'notes', 'first_name', 'middle_name', 'last_name'],
      dtype='object')

In [60]:
dfa = dfa[dfa.state.isin(["ma"])]
dfa.loc[:, "notes"] = dfa.notes.str.lower().str.strip()
dfa.loc[:, "first_name"] = dfa.first_name.str.replace(r"^ (\w+)", r"\1", regex=True).str.replace(r"(\w+) $", r"\1", regex=True)
dfa.loc[:, "last_name"] = dfa.last_name.str.replace(r"^ (\w+)", r"\1", regex=True).str.replace(r"(\w+) $", r"\1", regex=True)
dfa.loc[:, "city"] = dfa.city.str.lower().str.strip().str.replace(r"\,.+", "", regex=True)

In [61]:
dfa.state.unique()

array(['ma'], dtype=object)

In [62]:
search_1 = dfa[dfa.notes.str.contains("law enforcement")]
search_1

,id_number,group_name,bool,city,state,email,undefined_date,notes,first_name,middle_name,last_name


In [63]:
search_2 = dfa[dfa.notes.str.contains("police")]
search_2

,id_number,group_name,bool,city,state,email,undefined_date,notes,first_name,middle_name,last_name
4122,1043,Annual,No,hopkinton,ma,cjdcarving@gmail.com,,"civilian - brothers are ret. mil, policeman, m...",cecilia,del,gaudio
8197,6075,Annual,No,westport,ma,David.Bellavance@us.ngrid.com,,cpr/fa/aed instuctor if people want to be cert...,david,,bellavance
13353,11462,Annual,No,braintree,ma,beungood8@aol.com,1/6/2012,i have been a fulltime sworn police officer f...,john,,ouellette


In [64]:
search_3 = dfa[dfa.notes.str.contains("patrol")]
search_3.notes.unique()

array(['i am a sergeant supervising a patrol shift in falmouth, massachusetts.  i am a member of the sandwich american legion post 188 and a member of the leathernecks motorcycle club.'],
      dtype=object)

In [65]:
search_4 = dfa[dfa.notes.str.contains("officer")]
search_4.notes.unique()

array(["also worked as k9 officer, also worked with nd attorney general's drug enforcement unit",
       "cpr/fa/aed instuctor if people want to be certified i can provide that. first responder trained for medical if needed. the skills i've learned as a soldier and part time police officer.",
       'i have been  a fulltime sworn police officer for 16 years.  i am into cycling,hunting fishing,martial arts ,kayaking and cross country skiing'],
      dtype=object)

In [66]:
search_5 = dfa[dfa.notes.str.contains("swat")]
search_5

,id_number,group_name,bool,city,state,email,undefined_date,notes,first_name,middle_name,last_name


In [67]:
search_6 = dfa[dfa.notes.str.contains("explosive")]
search_6

,id_number,group_name,bool,city,state,email,undefined_date,notes,first_name,middle_name,last_name


In [68]:
search_7 = dfa[dfa.notes.str.contains("bomb")]
search_7

,id_number,group_name,bool,city,state,email,undefined_date,notes,first_name,middle_name,last_name


In [69]:
search_8 = dfa[dfa.notes.str.contains("active duty")]
search_8.notes.unique()

array(['spreading the word, and god forbid forming a neighborhood / town militia to keep the oath. i am the son of a us army msgt who died active duty (cid)1971, british boys brigade at 8yo,shot my first m60 at age of 9 at green berit day new cumberland army depot,attended west point youth camp 4 years.i know what kind of people can be counted on and trusted.and i will never submit to any enemy foriegn or domestic.'],
      dtype=object)

In [70]:
dfs = [search_1, search_2, search_3, search_4, search_5, search_6, search_7, search_8]
dfb = pd.concat(dfs, axis=0)

In [71]:
def read_misconduct_data():
    df1 = pd.read_csv("../data/MA/misconduct/nacdl/billerica/ia/uof_billerica_2015_2020.csv")
    df2 = pd.read_csv("../data/MA/misconduct/nacdl/Brookline/IIU/cprr_brookline_2010_2021.csv")
    df3 = pd.read_csv("../data/MA/misconduct/nacdl/Chelmsford/IA/cprr_chelmsford_2015_2016.csv")
    df4 = pd.read_csv("../data/MA/misconduct/nacdl/Dartmouth/IIU/cprr_dartmouth_2013_2022.csv")
    df5 = pd.read_csv("../data/MA/misconduct/nacdl/Framingham/IIU/cprr_framingham_2011_2021.csv")
    return df1, df2, df3, df4, df5

In [72]:
df1, df2, df3, df4, df5 = read_misconduct_data()

In [73]:
def split_names(df):
    df = df.pipe(clean_column_names)
    df = df[~((df.name.fillna("") == ""))]
    names = df.name.str.lower().str.strip()\
        .str.extract(r"(.+)\,? (\w+) ?(\w+)?")
    
    df.loc[:, "last_name"] = names[0]
    df.loc[:, "first_name"] = names[1]
    df.loc[:, "middle_name"] = names[2]
    return df

In [74]:
df1[["first_name", "middle_name"]] = ""

df2 = df2.pipe(split_names)

df3 = df3.pipe(clean_column_names)
df3 = df3.rename(columns={"name": "last_name"})
df3[["first_name", "middle_name"]] = ""

df4 = df4.pipe(split_names)

df5

# dfc = pd.concat([df1, df2, df3, df4], axis=0)
# dfc = dfc.reset_index()
# dfc = dfc[["first_name", "middle_name", "last_name"]]
# dfc = dfc.pipe(standardize_desc_cols, ["first_name", "middle_name", "last_name"])
# dfc

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,name,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,'2/17/2011 2011-01,'IAI,'N/A,'Lieutenant,"'DIVITTORIO, FRANK",'120,'SUSTAINED,'03/01/2011,'30 DAY SUSPENSION,'N CRIMINAL CONDUCT,'Threatening another officer,'Lt.,NaN,NaN
1,',',','PATROL OFFICER,"'COOTS, DOLORES",'114,'SUSTAINED,'03/24/2011,'LETTER OF COUNSELING,','Discourteous and Failed to complete police re...,',NaN,NaN
2,'1/25/2011 2011-02,'PSI,'CC2011-01,'LIEUTENANT,"'DELANEY, WILLIAM",'119,'SUSTAINED,'03/21/2011,'LETTER OF COUNSELING,'N COURTESY,'Failed to deliver effective and proper servic...,'Dep.,NaN,NaN
3,',',','PATROL OFFICER,"'MCCAN, MICHAEL",'160,'CLEARED,'03/23/2011,',',',',NaN,NaN
4,'1/26/2011 2011-03,'PSI,'CC2011-02,'PATROL OFFICER,'LEONARD PINI,'289,'CLEARED,'03/23/2011,'FILED,'N PROFESSIONAL IMAGE,'Discourteous Professional Image.,'Sgt.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,',',',',',',',',','Policy and Procedure Compliance,',',NaN,NaN
353,',',',',',',',',','Use of Department Vehicles,',',NaN,NaN
354,'11/30/2017,'2017-25 PSI,'CC2017-10,'SERGEANT,"'BROWN, SCOTT",'253,'CLEARED 01/20/2018,'FILE,'n,'Departmental Property and Equipment,'Unsafe operation of police vehicle and abuse ...,'Lt. Blaise Tersoni,NaN,NaN
355,',',',',',',',',',',',',NaN,NaN


In [75]:

def match_ok_names_to_personnel(ok, agency):
    dfa = ok[["first_name", "middle_name", "last_name", "notes", "city", "state"]]

    dfb = agency[["first_name", "middle_name", "last_name"]]
    dfb = dfb.fillna("")
    dfb["city"] = ""
    dfb["state"] = ""
    dfb["notes"] = ""

    matcher = ThresholdMatcher(
        ColumnsIndex([]),
        {
            "first_name": JaroWinklerSimilarity(),
            "last_name": JaroWinklerSimilarity(),
        },
        dfa,
        dfb,
    )
    decision = 0.800
    matcher.save_pairs_to_excel(
        ("../data/MA/oathkeepers_v_ma.xlsx"), decision
    )
    return matcher

In [76]:
match = match_ok_names_to_personnel(dfa, dfc)

KeyboardInterrupt: 

In [ ]:
def read_matches():
    df = pd.read_excel("../data/MA/oathkeepers_v_ma.xlsx", sheet_name=1)
    return df 

In [ ]:
matches = read_matches()

In [ ]:
matches.iloc[:50]

,pair_idx,sim_score,row_key,first_name,middle_name,last_name,notes,city,state
0,0.0,0.874922,3450,john,NaN,francolini,NaN,westborough,ma
1,NaN,NaN,249,john,NaN,"fallon, kevin",NaN,NaN,NaN
2,1.0,0.866364,8018,john,NaN,hatcher,local activities.,shrewsbury,ma
3,NaN,NaN,254,john,NaN,"hayes, derek",NaN,NaN,NaN
4,2.0,0.854180,36788,william,harrison,myron pruyne,NaN,east longmeadow,ma
5,NaN,NaN,265,william,NaN,"murphy, robert",NaN,NaN,NaN
6,3.0,0.840614,24269,david,NaN,heroux,NaN,north oxford,ma
7,NaN,NaN,304,david,NaN,"cheung,",NaN,NaN,NaN
8,4.0,0.838597,15846,francis,j,"nash, jr.",NaN,scituate,ma
9,NaN,NaN,244,francis,NaN,"jennings, john",NaN,NaN,NaN
